# Model Evaluation And Validation

## Tutorial Naive Bayes

The Naive Bayes documentation for scikit learn can be found [here](http://scikit-learn.org/stable/modules/naive_bayes.html)

### Example: sklearn.naive_bayes.GaussianNB

The example code is taken from [here](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html])

In [1]:
import numpy as np

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])

from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X, Y)

print clf.predict([[-0.8, -1]])

clf_pf = GaussianNB()
clf_pf.partial_fit(X, Y, np.unique(Y))

print clf_pf.predict([[-0.8, -1]])

[1]
[1]


## Evaluation Metrics

Some metrics for performance in classification models are used here.

### Accuracy
SciKit learn documentation [here](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)

In [2]:
from sklearn.metrics import accuracy_score

y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]

print accuracy_score(y_true, y_pred)

print accuracy_score(y_true, y_pred, normalize=False)

0.5
2


### F1 Score

SciKit learn documentation [here](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)

In [3]:
from sklearn.metrics import f1_score

y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]

print f1_score(y_true, y_pred, average='macro')  

print f1_score(y_true, y_pred, average='micro')  

print f1_score(y_true, y_pred, average='weighted')  

print f1_score(y_true, y_pred, average=None)

0.266666666667
0.333333333333
0.266666666667
[ 0.8  0.   0. ]


## Cross-Validation

### Simple train test cross validation

This example borrows heavily from the example shown on the SciKit learn documentation [here](http://scikit-learn.org/stable/modules/cross_validation.html)

In [4]:
#!/usr/bin/python

from sklearn import datasets
from sklearn.svm import SVC

iris = datasets.load_iris()
features = iris.data
labels = iris.target

###############################################################
### YOUR CODE HERE
###############################################################

### import the relevant code and make your train/test split
### name the output datasets features_train, features_test,
### labels_train, and labels_test

### set the random_state to 0 and the test_size to 0.4 so
### we can exactly check your result

from sklearn import cross_validation

features_train, features_test, labels_train, labels_test \
= cross_validation.train_test_split(features, labels, test_size=0.4, random_state=0)

###############################################################

clf = SVC(kernel="linear", C=1.)
clf.fit(features_train, labels_train)

print clf.score(features_test, labels_test)


##############################################################
def submitAcc():
    return clf.score(features_test, labels_test)

0.966666666667


### K-Fold Cross Validation

With simple train test cross validation one of the problems one has is deciding how much data to use for training versus testing. K_Fold cross validation is a way to use all our data for both training and testing.

The general idea is that you partition your data into k equal parts and run k separate learning experiments. For each experiment you pick a different testing set and train on the remaining data. You then average test results from those k experiments.

Clearly this method is computationally more expensive than the simple train test method but yields a more accurate model.

One of the problems which can arise with this method in SKLearn is that the data is partitioned sequentially. Then you have to be careful that there is a good mix of examples in each partition else you might be training on one type of example and then testing on another. There's a simple way to randomize the events in sklearn k-fold CV: set the shuffle flag to true.

Instead of something like this:

    cv = KFold( len(authors), 2 )

one could write:

    cv = KFold( len(authors), 2, shuffle=True )

### Grid Cross Validation

GridCV is a way of systematically working through multiple combinations of parameter tunes, cross-validating as it goes to determine which tune gives the best performance. The beauty is that it can work through many combinations in only a couple extra lines of code.

Here's an example from the sklearn documentation, which can be found [here](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html):

```
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svr = svm.SVC()
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(iris.data, iris.target)
```

Let's break this down line by line.

    parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]} 

A dictionary of the parameters, and the possible values you want to try for them. In this case, they're playing around with the kernel (possible choices are 'linear' and 'rbf'), and C (possible choices are 1 and 10).

Then all the following (kernel, C) combinations are automatically generated: [('rbf', 1), ('rbf', 10), ('linear', 1), ('linear', 10)]. Each is used to train an SVM, and the performance is then assessed using cross-validation.

    svr = svm.SVC()

This looks kind of like creating a classifier, just like we've been doing since the first lesson. But note that the "clf" isn't made until the next line--this is just saying what kind of algorithm to use. Another way to think about this is that the "classifier" isn't just the algorithm in this case, it's algorithm plus parameter values. Note that there's no monkeying around with the kernel or C; all that is handled in the next line.

    clf = grid_search.GridSearchCV(svr, parameters)

This is where the first bit of magic happens; the classifier is being created. We pass the algorithm (svr) and the dictionary of parameters to try (parameters) and it generates a grid of parameter combinations to try.

    clf.fit(iris.data, iris.target)

And the second bit of magic. The fit function now tries all the parameter combinations, and returns a fitted classifier that's automatically tuned to the optimal parameter combination. You can now access the parameter values via clf.best_estimator_.